In [7]:
import yfinance as yf
import pandas as pd
import numpy as np

In [8]:
# Define the list of indices and equities
indices = ["^GSPC", "^DJI", "^IXIC", "^FTSE", "^N225"]
equities = ["AAPL", "GOOGL", "MSFT", "TSLA", "AMZN"]

In [9]:
# Load the daily data for indices
index_data = yf.download(indices, start='2010-01-01')['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [10]:
# Create an empty DataFrame to store equity data
equity_data = pd.DataFrame()

# Retrieve data for each equity
for equity in equities:
    equity_ticker = yf.Ticker(equity)
    equity_history = equity_ticker.history(start='2010-01-01')
    equity_data[equity] = equity_history['Close']

# Convert equity_data to time zone-naive
equity_data = equity_data.tz_localize(None)

# Combine index and equity data
data = pd.concat([index_data, equity_data], axis=1)

# Calculate daily returns
returns = data.pct_change().dropna()

# Calculate cumulative returns
cumulative_returns = (1 + returns).cumprod()

# Calculate maximum drawdowns
roll_max = cumulative_returns.rolling(window=len(cumulative_returns), min_periods=1).max()
daily_drawdown = cumulative_returns / roll_max - 1.0
max_drawdown = daily_drawdown.min()

# Calculate volatility
volatility = returns.std() * np.sqrt(252)

# Calculate the Sharpe ratio
risk_free_rate = 0.0  # Assuming no risk-free rate for simplicity
daily_risk_free_rate = (1 + risk_free_rate) ** (1/252) - 1
excess_returns = returns - daily_risk_free_rate
sharpe_ratio = np.mean(excess_returns) / np.std(returns) * np.sqrt(252)

# Calculate the Sortino ratio
downside_returns = returns.copy()
downside_returns[downside_returns > 0] = 0
sortino_ratio = np.mean(excess_returns) / np.std(downside_returns) * np.sqrt(252)

In [11]:
# Create tables for indices and equities
index_metrics = pd.DataFrame({'Cumulative Returns': cumulative_returns.iloc[-1],
                              'Volatility': volatility,
                              'Max Drawdown': max_drawdown,
                              'Sharpe Ratio': sharpe_ratio,
                              'Sortino Ratio': sortino_ratio},
                             index=indices)

equity_metrics = pd.DataFrame({'Cumulative Returns': cumulative_returns.iloc[-1, len(indices):],
                               'Volatility': volatility[len(indices):],
                               'Max Drawdown': max_drawdown[len(indices):],
                               'Sharpe Ratio': sharpe_ratio[len(indices):],
                               'Sortino Ratio': sortino_ratio[len(indices):]},
                              index=equities)

# Display the tables
print("Index Metrics:")
print(index_metrics)
print("\nEquity Metrics:")
print(equity_metrics)


Index Metrics:
       Cumulative Returns  Volatility  Max Drawdown  Sharpe Ratio   
^GSPC            4.025950    0.173473     -0.339250      1.193479  \
^DJI             3.386587    0.168469     -0.370862      1.228930   
^IXIC            5.928259    0.201898     -0.363953      1.025449   
^FTSE            1.578466    0.158791     -0.366055      1.303829   
^N225            3.219038    0.201278     -0.317989      1.028608   

       Sortino Ratio  
^GSPC       1.841189  
^DJI        1.884406  
^IXIC       1.601719  
^FTSE       2.023831  
^N225       1.642336  

Equity Metrics:
       Cumulative Returns  Volatility  Max Drawdown  Sharpe Ratio   
AAPL            22.525537    0.279898     -0.437972      0.739686  \
GOOGL           10.798858    0.268601     -0.443201      0.770796   
MSFT            17.767446    0.257479     -0.371485      0.804091   
TSLA           113.105879    0.563767     -0.736322      0.367238   
AMZN            21.406868    0.325373     -0.561453      0.636306   



In [12]:
index_metrics.to_csv('./Index.csv')
equity_metrics.to_csv('./Equity.csv')

In [17]:
index_metrics = pd.read_csv('./Index.csv')
display(index_metrics)


equity_metrics = pd.read_csv('./Equity.csv')
display(equity_metrics)

,Unnamed: 0,Cumulative Returns,Volatility,Max Drawdown,Sharpe Ratio,Sortino Ratio
0,^GSPC,4.025950,0.173473,-0.339250,1.193479,1.841189
1,^DJI,3.386587,0.168469,-0.370862,1.228930,1.884406
2,^IXIC,5.928259,0.201898,-0.363953,1.025449,1.601719
3,^FTSE,1.578466,0.158791,-0.366055,1.303829,2.023831
4,^N225,3.219038,0.201278,-0.317989,1.028608,1.642336


,Unnamed: 0,Cumulative Returns,Volatility,Max Drawdown,Sharpe Ratio,Sortino Ratio
0,AAPL,22.525537,0.279898,-0.437972,0.739686,1.229321
1,GOOGL,10.798858,0.268601,-0.443201,0.770796,1.296539
2,MSFT,17.767446,0.257479,-0.371485,0.804091,1.349842
3,TSLA,113.105879,0.563767,-0.736322,0.367238,0.634086
4,AMZN,21.406868,0.325373,-0.561453,0.636306,1.077675
